In [1]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [2]:
cd "/content/drive/Othercomputers/My Laptop/crypto_prediction_notebooks/"

/content/drive/Othercomputers/My Laptop/crypto_prediction_notebooks


In [3]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.8 MB/s eta 0:00:00


In [6]:
_RANDOM_STATE = 42

In [7]:
import pandas as pd

# Load the model from crypto_history.csv
crypto_history_df = pd.read_csv('../datasets/crypto_history.csv')


In [8]:
crypto_history_df

,dateTime,open,high,low,close,volume
0,1673906400000,21133.34,21224.12,21127.04,21196.57,3382.46830
1,1673908200000,21198.21,21210.57,21142.06,21185.89,2401.86142
2,1673910000000,21185.89,21227.00,21097.27,21211.51,3753.32135
3,1673911800000,21211.51,21218.26,21151.47,21185.65,2403.98364
4,1673913600000,21185.65,21299.04,21070.01,21220.08,8626.68241
...,...,...,...,...,...,...
17509,1705426200000,43187.98,43187.99,43031.00,43051.74,610.79146
17510,1705428000000,43051.74,43157.85,42928.71,42965.66,695.19227
17511,1705429800000,42965.65,43050.00,42834.29,43013.00,842.31440
17512,1705431600000,43013.00,43158.00,42949.00,43101.11,575.48109


In [9]:
# Convert the Unix timestamp to datetime
crypto_history_df['dateTime'] = pd.to_datetime(crypto_history_df['dateTime'], unit='ms')

In [10]:
from sklearn.preprocessing import MinMaxScaler
import pickle

# Normalize the data
scaler = MinMaxScaler(feature_range=(0,1))
crypto_history_data = scaler.fit_transform(crypto_history_df[['close']])

# Save the scaler
with open('../models/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [13]:
import numpy as np

def processData(data, lb):
    X, Y = [], []
    for i in range(len(data) - lb - 1):
        X.append(data[i: (i + lb), 0])
        Y.append(data[(i + lb), 0])
    return np.array(X), np.array(Y)

# 7 days of data when each day has 48 records (for every 30 minutes)
X, y = processData(crypto_history_data, 7 * 48)

((17177, 336), (17177,))

In [10]:
from sklearn.model_selection import train_test_split

# Assuming 'X' and 'y' are your processed data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=_RANDOM_STATE)


# Reshape the data to be 3-dimensional (samples, timesteps, features)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [11]:
import tensorflow as tf
from tensorflow import keras
from kerastuner.tuners import RandomSearch

def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.LSTM(units=hp.Int('units_1', min_value=32, max_value=512, step=32),
                          input_shape=(X_train.shape[1], 1), return_sequences=True))
    for i in range(1, hp.Int('num_layers', 2, 5) - 1):
        model.add(keras.layers.LSTM(units=hp.Int('units_' + str(i+1), min_value=32, max_value=512, step=32), return_sequences=True))
    model.add(keras.layers.LSTM(units=hp.Int('units_' + str(hp.Int('num_layers', 2, 5)), min_value=32, max_value=512, step=32)))
    model.add(keras.layers.Dense(1))
    model.compile(
        optimizer=keras.optimizers.Adam(
            learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error')
    return model

<ipython-input-11-1c96c7dab129>:3: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [12]:
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,
    executions_per_trial=3,
    directory='models',
    project_name='crypto_prediction')

In [13]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    verbose=1,
    patience=10,
    mode='min',
    restore_best_weights=True)

In [ ]:
tuner.search_space_summary()

tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test),
             callbacks=[early_stopping])

Search space summary
Default search space size: 4
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 5, 'step': 1, 'sampling': 'linear'}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}

Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
224               |224               |units_1
3                 |3                 |num_layers
64                |64                |units_2
0.01              |0.01              |learning_rate

Epoch 1/5
430/430 [==============================] - 26s 43ms/step - loss: 0.1577 - val_loss: 0.1522
Epoch 2/5
430/430 [==============================] - 18s 41ms/step - loss: 0.1522 - val_loss: 0.1499
Epoch 3/5

In [17]:
best_model = tuner.get_best_models(num_models=1)[0]

ValueError: Received incompatible tensor with shape (1408,) when attempting to restore variable with shape (1152,) and name lstm/lstm_cell/bias:0.

In [ ]:
from sklearn.metrics import mean_squared_error

# Predict the values with the best model
y_pred = best_model.predict(X_test)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error: ", mse)


In [16]:
# Specify the path where you want to save the model
model_path = "../models/crypto_prediction.h5"

# Save the model
best_model.save(model_path)

NameError: name 'best_model' is not defined